In [36]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/selinawang/nltk_data...


True

In [16]:
import tensorflow as tf
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
import pickle
import os

In [13]:
with open('../data/pg1597.txt') as f:
    lines = f.readlines()
print(lines[:6])

["THE EMPEROR'S NEW CLOTHES\n", '\n', 'Many years ago, there was an Emperor, who was so excessively fond of\n', 'new clothes, that he spent all his money in dress. He did not trouble\n', 'himself in the least about his soldiers; nor did he care to go either to\n', 'the theatre or the chase, except for the opportunities then afforded him\n']


In [14]:
with open('../data/78.txt') as f:
    lines += f.readlines()

In [23]:
lines

["THE EMPEROR'S NEW CLOTHES\n",
 '\n',
 'Many years ago, there was an Emperor, who was so excessively fond of\n',
 'new clothes, that he spent all his money in dress. He did not trouble\n',
 'himself in the least about his soldiers; nor did he care to go either to\n',
 'the theatre or the chase, except for the opportunities then afforded him\n',
 'for displaying his new clothes. He had a different suit for each hour of\n',
 'the day; and as of any other king or emperor, one is accustomed to say,\n',
 '“he is sitting in council,” it was always said of him, “The Emperor is\n',
 'sitting in his wardrobe.”\n',
 '\n',
 'Time passed merrily in the large town which was his capital; strangers\n',
 'arrived every day at the court. One day, two rogues, calling themselves\n',
 'weavers, made their appearance. They gave out that they knew how to\n',
 'weave stuffs of the most beautiful colors and elaborate patterns, the\n',
 'clothes manufactured from which should have the wonderful property of\n'

In [5]:
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

def remove_punctuation(text):
    punct_list = list('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~“”')
    for punc in punct_list:
        if punc in text:
            text = text.replace(punc, '')
    return text.strip()

In [35]:
def preprocess(line):
    data = "".join(line)
    data = word_tokenize(data)
    words = " ".join(data)
    lower_w = words.lower()
    
    # keep gender pronouns in stop words
    stop_words_keep = ['he',
     'him',
     'his',
     'himself',
     'she',
     "she's",
     'her',
     'hers',
     'herself']
    stop_words = [word for word in stopwords.words('english') if word not in stop_words_keep]

    stop_words = set(stop_words) 
    word_tokens = lower_w.split(" ")
  
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = " ".join(filtered_sentence)
    nopunct_sentence = remove_punctuation(filtered_sentence)
    nopunct_sentence = nopunct_sentence.split(' ')
    #print("before lemmatization:", nopunct_sentence)
    lemmatized_sentence = lemmatizer(nopunct_sentence)
    #print("after lemmatization:", lemmatized_sentence)
    
#     ps = PorterStemmer()
#     output = []
#     for word in filtered_sentence:
#         output.append((ps.stem(word)))
#     output = " ".join(output)
    
    #final_out = lemmatized_sentence.replace('\n', ' ').split(' ')
    return lemmatized_sentence

In [36]:
def get_vocab(data):
    words = list(set(data))
    vocabulary = {word:index for index, word in enumerate(words)}
    vocab_size = len(vocabulary)
    
    return vocabulary, vocab_size

In [37]:
train_data = preprocess(lines)
vocabulary, vocab_size = get_vocab(train_data)

In [38]:
train_data

['emperor',
 's',
 'new',
 'clothes',
 'many',
 'year',
 'ago',
 '',
 'emperor',
 '',
 'excessively',
 'fond',
 'new',
 'clothes',
 '',
 'he',
 'spent',
 'his',
 'money',
 'dress',
 '',
 'he',
 'trouble',
 'himself',
 'least',
 'his',
 'soldier',
 '',
 'he',
 'care',
 'go',
 'either',
 'theatre',
 'chase',
 '',
 'except',
 'opportunity',
 'afforded',
 'him',
 'displaying',
 'his',
 'new',
 'clothes',
 '',
 'he',
 'different',
 'suit',
 'hour',
 'day',
 '',
 'king',
 'emperor',
 '',
 'one',
 'accustomed',
 'say',
 '',
 '',
 'he',
 'sitting',
 'council',
 '',
 '',
 'always',
 'said',
 'him',
 '',
 '',
 'emperor',
 'sitting',
 'his',
 'wardrobe',
 '',
 'time',
 'passed',
 'merrily',
 'large',
 'town',
 'his',
 'capital',
 '',
 'stranger',
 'arrived',
 'every',
 'day',
 'court',
 '',
 'one',
 'day',
 '',
 'two',
 'rogue',
 '',
 'calling',
 'weaver',
 '',
 'made',
 'appearance',
 '',
 'gave',
 'knew',
 'weave',
 'stuff',
 'beautiful',
 'color',
 'elaborate',
 'pattern',
 '',
 'clothes',
 'm

In [9]:
#dictionary of word counts in text
wordFreqDict={}
for word in train_data:
    if word in wordFreqDict:
        wordFreqDict[word]=wordFreqDict[word]+1
    else:
        wordFreqDict[word]=1


In [10]:
# sort dictionary 
wordFreqDict = dict(sorted(wordFreqDict.items(), key=lambda item: item[1], reverse=True))

In [11]:
# create a new vocabulary with the top 5000 words
vocabulary_5000 = [v for v in list(wordFreqDict.keys())[:5000]]
vocabulary_5000.append('<UNK>')

In [29]:
# do preprocessing for all of the data:
data_path = '../data'
texts = []
for root, dirs, files in os.walk(data_path):
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'r') as f:
                text = f.readlines()
                texts += text


In [39]:
len(texts)

335940

In [40]:
preprocessed_texts = preprocess(texts)

In [42]:
len(preprocessed_texts)

2206682

In [43]:
# keep only the top 5000 words
preprocessed_texts_5000 = [word if word in vocabulary_5000 else '<UNK>' for word in preprocessed_texts]
preprocessed_texts_5000

['<UNK>',
 'gutenberg',
 '<UNK>',
 'welsh',
 '<UNK>',
 'story',
 '',
 '<UNK>',
 'ebook',
 'use',
 'anyone',
 'anywhere',
 'cost',
 'almost',
 'restriction',
 'whatsoever',
 '',
 'may',
 'copy',
 '',
 'give',
 'away',
 'reuse',
 'term',
 'project',
 'gutenberg',
 'license',
 'included',
 'ebook',
 'online',
 'wwwgutenbergorg',
 'title',
 '',
 'welsh',
 '<UNK>',
 'story',
 'author',
 '',
 '<UNK>',
 'editor',
 '',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 'release',
 'date',
 '',
 '<UNK>',
 '',
 '<UNK>',
 '',
 'ebook',
 '',
 '<UNK>',
 '',
 'file',
 'first',
 'posted',
 '<UNK>',
 '<UNK>',
 '',
 '<UNK>',
 'last',
 '<UNK>',
 '',
 '<UNK>',
 '<UNK>',
 '',
 '<UNK>',
 'language',
 '',
 'english',
 'character',
 'set',
 '<UNK>',
 '',
 '<UNK>',
 '',
 '',
 '',
 'start',
 'project',
 'gutenberg',
 'ebook',
 'welsh',
 '<UNK>',
 '',
 '',
 '',
 'produced',
 '<UNK>',
 '<UNK>',
 '',
 'charles',
 '<UNK>',
 'people',
 '<UNK>',
 'welsh',
 '<UNK>',
 'story',
 '<UNK>',
 'collected',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>

In [48]:
with open('../preprocessed_texts.txt', 'w') as fp:
    for text in preprocessed_texts_5000:
        # write each item on a new line
        fp.write('%s '%text)

In [55]:
vocabulary, vocab_size = get_vocab(preprocessed_texts_5000)


In [57]:
vocabulary

{'': 0,
 'thornbush': 1,
 'heaven': 2,
 'haberdine': 3,
 'useless': 4,
 'summit': 5,
 'lit': 6,
 'glad': 7,
 'echo': 8,
 'goodtempered': 9,
 'rambin': 10,
 'revenge': 11,
 'cart': 12,
 'held': 13,
 'manufacture': 14,
 'repulsive': 15,
 'triangle': 16,
 'odour': 17,
 'amiable': 18,
 'condescension': 19,
 'liked': 20,
 'gratitude': 21,
 'snore': 22,
 'afterwards': 23,
 'lifetime': 24,
 'return': 25,
 'forehead': 26,
 'entirely': 27,
 'victorious': 28,
 'shift': 29,
 'disguise': 30,
 'polished': 31,
 'attendant': 32,
 'executed': 33,
 'bravery': 34,
 'blackguard': 35,
 'plan': 36,
 'must': 37,
 'flutter': 38,
 'bedchamber': 39,
 'zelia': 40,
 'jasmine': 41,
 'resume': 42,
 'gailycolored': 43,
 'trickled': 44,
 'shower': 45,
 'dramatic': 46,
 'especially': 47,
 'protest': 48,
 'adder': 49,
 'cease': 50,
 'alexandria': 51,
 'pondered': 52,
 'method': 53,
 'necklace': 54,
 'hit': 55,
 'share': 56,
 'exist': 57,
 'lung': 58,
 'amused': 59,
 'grating': 60,
 'lord': 61,
 'small': 62,
 'viswanat

In [58]:
# save vocabulary
with open('../vocabulary.pkl', 'wb') as fp:
    pickle.dump(vocabulary, fp)